<a href="https://colab.research.google.com/github/RyanTahnikoyev/windows-97-robidatathon/blob/main/robi_data_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import glob
import math
from tensorflow.keras.applications import resnet50
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras import layers
import pandas as pd 

In [ ]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/robi-datathon-2-pre-assessment/train.csv")
# check the raw data
print("Size of the train dataset (train_data, col): ", train_data.shape)

test_data = pd.read_csv("/content/drive/MyDrive/robi-datathon-2-pre-assessment/test.csv")
# check the raw data
print("Size of the test dataset (test_data, col): ", test_data.shape)